In [ ]:
# https://colab.research.google.com/drive/1WbAd7ZuQNZXb2I1mZSjaXYUagcvpi889#scrollTo=2eDIEy2-wG9b&uniqifier=1

In [ ]:
import copy

In [ ]:
def hash_list(c1, c2):
  return "cols: " + str(c1) + " | vals: " + str(c2) 

def get_val_fact(map, cols, vals):
  return map[hash_list(cols, vals)]

class dataF:

  def __init__(self,n_i,names,df_names,ed=[]):

    self.n_i = n_i
    self.names = names
    self.n = len(self.names)
    self.uniq = [set() for x in range(self.n)]
    self.df_names = df_names
    
    # uniq values
    for i in range(self.n_i):
      row = df_names[i]
      for j in range(self.n):
        self.uniq[j].add(row[j])

    self.hiper_param = 1
    
    #mapear elementos
    self.mapElems()

    # get ad and in list
    self.lad = [[] for x in range(self.n)] 
    self.lin = [[] for x in range(self.n)]

    for elem in ed:
      u = self.mnames[elem[0]]
      v = self.mnames[elem[1]]
      self.lad[u].append(v)
      self.lin[v].append(u)

    self.initParents()
    self.getFactors()

    self.edges = []
    for i in range(self.n):
      u = self.names[i]
      for x in self.lad[i]:
        v = self.names[x]
        self.edges.append([u, v])
  
  def mapElems(self):
    # map columns
    self.card = []
    self.mnames = dict()
    for x in range(self.n):
      self.mnames[self.names[x]] = x
    
    self.mcvals = [dict() for x in range(self.n)]
    for c in range(self.n):
      self.uniq[c] = list(self.uniq[c])
      self.card.append(len(self.uniq[c]))
      for i in range(self.card[c]):
        self.mcvals[c][self.uniq[c][i]] = i
    
    self.df = []
    for row in self.df_names:
      aux = []
      for c in range(self.n):
        aux.append(self.mcvals[c][row[c]])
      
      self.df.append(aux)

  def initParents(self):  
    self.parent = [-1]*self.n
    flag = [0]*self.n

    def dfs(u, p, vis):
      if(vis[u]):
        return 
      
      vis[u] = True
      self.parent[u] = p
      for x in self.lad[u]:
        dfs(x, u, vis) 

    self.root = -1
    for x in range(self.n):
      for y in self.lad[x]:
        flag[y] = 1
    
    for x in range(self.n):
      if(not flag[x]):
        self.root = x
        break

    vis = [False]*self.n
    dfs(self.root, -1, vis)

  def getFactors(self):
    self.mapFact = dict()
    for i in range(self.n):
      cols = [i] + self.lin[i] 
      fact = self.calcFactor(i)
      for x in fact:
        self.mapFact[hash_list(cols, x[:-1])] = x[-1]

  def reset(self):
    self.initParents()
    self.getFactors()

  def rep(self, params):
    cnt = 0
    for row in self.df:
      ok = True
      for x in params:
        if(row[x[0]] != x[1]):
            ok = False
      if(ok):
        cnt+= 1
      
    return cnt

  def prob_general(self, params, factor):
    # (m[value] + p) / (M + (p * card(column))
    exp1 = self.rep(params) + self.hiper_param
    exp2 = self.n_i + (self.hiper_param * factor)
    return round(exp1/exp2,6) 
      
  def calcFactor(self, id):

    def normalize(arr):
      s = 0
      ans = []

      for x in arr:
        s+= x
      
      for x in arr:
        ans.append(x/s)
      
      return ans
      
    def add_column(M, col):
      for i in range(len(col)):
        M[i].append(col[i])

    cols = [id] + self.lin[id]

    uniqs = []
    cnt = []
    ids = []
    vuelta = []
    total = 1

    for x in cols:

        uniqs.append([i for i in range(self.card[x])])
        ids.append(total)
            
        total*= self.card[x]
        
        cnt.append(0)
        vuelta.append(0)

    D = []
    c_ans = []
    c_norm = []
    M = []

    for u in range(total):
        vals = []
        i = 0
        for x in cols:
            vals.append([x, uniqs[i][cnt[i]]])
            i+= 1
        
        M.append( [e[1] for e in vals] )

        exp1 = self.prob_general(vals, total)
        exp2 = self.prob_general(vals[1:], total/self.card[id])

        if(len(self.lin[id]) == 0):
          D.append(exp1)
        
        else:
          D.append(exp1/exp2)

        for j in range(len(cols)):
            vuelta[j]+= 1
            if(vuelta[j] == ids[j]):
                cnt[j] = (cnt[j] + 1)%len(uniqs[j])
                vuelta[j] = 0

        if(len(D) == self.card[id]):
          c_ans+= D
          c_norm+= normalize(D)
          D = []
          
    # Agregando columnas
    add_column(M, c_norm)

    return M


def Entropia(D):
  ans = 0
  for i in range(D.n):
    combs = D.calcFactor(i)
    for x in range(len(combs)):
      combs[x] = combs[x][:-1]

    for comb in combs:
      params = [[i, comb[0]]]
      for id in range(1, len(comb)):
        params.append([ D.lin[i][id - 1] , comb[id]])
      

      Nijk = D.rep(params)
      a = Nijk/D.n_i
      
      params = params[1:]
      den = D.n_i
      
      if(params):
        den = D.rep(params)

      if(Nijk == 0):
        continue

      b = math.log2(Nijk/den)

      ans+= a*b
  
  return -D.n_i*ans

def AIC(D):
  K = 0
  for i in range(D.n):
    q = 1
    for x in D.lin[i]:
      q *= D.card[x]

    K+= (D.card[i]-1)*q
  
  return Entropia(db) + K

def MDL(D):
  K = 0
  for i in range(D.n):
    q = 1
    for x in D.lin[i]:
      q *= D.card[x]

    K+= (D.card[i]-1)*q
  
  return Entropia(db) + K*math.log2(D.n_i)/2.0

def hk2(D):

  def fact(num):
    ans = 1
    while(num > 0):
      ans*= num
      num-= 1
    
    return ans

  ans = 1
  for i in range(D.n):
    P = D.parent[i]
    if(P != -1):
      for j in range(D.card[P]):
        Nij = D.rep([ [ P , j ] ] )
        term = 1
        for k in range(D.card[i]):
          Nijk = D.rep([ [P, j] , [i , k ] ])
          term*= fact(Nijk)

        
        term*= fact(D.card[i] - 1)/fact(D.card[i] - 1 + Nij)
        
        ans*= term

  return ans

def K2(D, lb, ub, metrica, order=None):

  def getEdges(nodes_parents, i=None, parents_i={}):
    ed = []

    for x in nodes_parents:
      for p in nodes_parents[x]:
        ed.append([p,x])
    
    if(i != None):
      for p in parents_i:
        ed.append([p, i])

    return ed


  nodes = D.names if order is None else order
  
  nodes_parents = {x: [] for x in nodes}
  scores = {'AIC': AIC, 'Entropy': Entropia, 'MDL': MDL, 'K2': hk2}


  for i in range(len(nodes)):
      parents_i = []
      
      P_old = scores[metrica](dataF(D.n_i, D.names, D.df_names, getEdges(nodes_parents, nodes[i], parents_i)))
      OKToProceed = True

      while OKToProceed and len(parents_i) < ub and i > 0:
        pred = [ nodes[ii] for ii in range(i)]

        maxs = []

        for node in pred:
          maxs.append(scores[metrica](dataF(D.n_i, D.names, D.df_names, getEdges(nodes_parents, nodes[i], parents_i + [node]))))
    
        mx = max(maxs)
        z = pred[maxs.index(mx)]
        P_new = mx

        if P_new > P_old:
          P_old = P_new
          parents_i.append(z)

        else:
          if(len(parents_i) < lb):
            parents_i.append(z)

          OKToProceed = False

      nodes_parents[nodes[i]] = parents_i
      # print("Nodo: ", nodes[i], " padres de Nodo => ", parents_i)

  return dataF(D.n_i, D.names, D.df_names, getEdges(nodes_parents))

def greedyLS(D, metrica):

  def cycle(u, G, vis, reStack):
    if(not vis[u]):
      vis[u] = True
      reStack[u] = True

      for x in G.lad[u]:
        if(not vis[x] and cycle(x, G, vis, reStack)):
          return True
        elif(reStack[x]):
          return True
      
    reStack[u] = False
    return False

  def isValid(G):
    vis = [False]*G.n
    reStack = [False]*G.n 

    for i in range(G.n):
      if(cycle(i, G, vis, reStack)):
        return False
    
    return True
    
  # Extrayendo lista de nodos
  nodes = D.names
    
  # Scores
  scores = {'AIC': AIC, 'Entropy': Entropia, 'MDL': MDL, 'K2': hk2}
 
  # 1. σbest ← σi
  omega_best = copy.deepcopy(D)
  score_best = scores[metrica](omega_best)

  # 2. do
  while True:
      
    # 3. σ ← σbest
    O = copy.deepcopy(omega_best)

    # 4.
    progress = False

    # 5. for each operator o ∈ O
    for node in nodes:
          
      cases = []
      
      for edge in O.edges:
        aux = [e for e in O.edges if e != edge]
        cases.append(dataF(O.n_i, O.names, O.df_names, aux))
        cases.append(dataF(O.n_i, O.names, O.df_names, aux + [[edge[1], edge[0]]]))
          
      for node2 in nodes:
        if([node, node2] not in O.edges and node != node2):
          cases.append(dataF(O.n_i, O.names, O.df_names, O.edges + [[node, node2]] ))         

      for case in cases:
        if isValid(case):
          score_o = scores[metrica](case)
          if score_o > score_best:
            omega_best = copy.deepcopy(case)
            score_best = score_o
            progress = True
 
    if not progress:
      break
    
  return omega_best

**Bag of Words**

In [ ]:
import math
import nltk
from nltk.tokenize import word_tokenize
import spacy
import re

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
spacy.cli.download('es_core_news_md')

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [ ]:
nlp = spacy.load('es_core_news_md')

In [ ]:
stopwords = nlp.Defaults.stop_words
print(stopwords)

{'grandes', 'pesar', 'hoy', 'tras', 'estais', 'indicó', 'raras', 'anterior', 'cierto', 'cuenta', 'demás', 'aquellas', 'usa', 'había', 'tuya', 'breve', 'cierta', 'explicó', 'cuantos', 'consiguen', 'asi', 'dan', 'misma', 'eras', 'ciertas', 'vaya', 'acuerdo', 'través', 'temprano', 'les', 'poco', 'ya', 'veces', 'junto', 'como', 'hecho', 'hubo', 'esta', 'señaló', 'todavía', 'dentro', 'trata', 'partir', 'valor', 'haya', 'afirmó', 'puedo', 'segunda', 'usas', 'he', 'nosotras', 'sido', 'vuestro', 'podria', 'antano', 'quiza', 'días', 'debajo', 'empleo', 'pero', 'hacer', 'ese', 'conmigo', 'casi', 'pronto', 'serán', 'hizo', 'habían', 'otros', 'tampoco', 'pasado', 'última', 'aqui', 'su', 'intentas', 'tiempo', 'usar', 'tú', 'dice', 'cualquier', 'gran', 'siempre', 'quien', 'quiere', 'vuestros', 'uno', 'poder', 'creo', 'si', 'mios', 'cuánto', 'vosotras', 'dijeron', 'le', 'apenas', 'vez', 'tanto', 'os', 'aquél', 'próximos', 'hacia', 'pocos', 'quién', 'último', 'primera', 'podeis', 'ciertos', 'últimos',

In [ ]:
def add_stop_word(stopword):
  nlp.vocab[stopword].is_stop = True
  stopwords.add(stopword)

In [ ]:
import pandas as pd
import re
from unicodedata import normalize


def process_text(text):
  
  def filter_non_an(X):
    PERMITTED_CHARS = "0123456789abcdefghijklmnñopqrstuvwxyzABCDEFGHIJKLMNÑOPQRSTUVWXYZ "   
    ans = ""
    for x in X:
      if(x in PERMITTED_CHARS):
          ans+=x
      else:
        ans+=" "

    return ans

  if(not isinstance(text, str)):
    return ""

  aux =  ' '.join(x for x in text.split() if not x.startswith('@'))
  aux = aux.lower()

  # -> NFD y eliminar diacríticos
  aux = re.sub(
          r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
          normalize( "NFD", aux), 0, re.I
      )

  # -> NFC
  aux = normalize( 'NFC', aux)

  aux = filter_non_an(aux)
  aux = re.sub(' +', ' ', aux)
  
  return aux

def readFile(file, lim = 50):
  df = pd.read_excel (file)

  names = df.columns
  df_names = []

  pos = []
  neu = []
  neg = []

  for text in df.values.tolist():
    
    aux = process_text(text[0])

    if(len(aux) > 0 and not aux.isspace()):
      if(text[1] == 'Positivo'):
        pos.append([aux, text[1]])
      elif(text[1] == 'Neutro'):
        neu.append([aux, text[1]])
      else:
        neg.append([aux, text[1]])
  

  part = int(min(lim/3, len(pos), len(neu), len(neg)))
  df_names = pos[:part] + neu[:part] + neg[:part] 

  return len(df_names), names, df_names

In [ ]:
import random

def get_filtered_df(df, n_i, max_nodes = 500):

  ls = []
  
  for X,Y in df:
    ls+= X.split(' ')
  
  unique_words = list(set(ls))
  random.shuffle(unique_words)


  print(unique_words)

  dif = len(unique_words) - max_nodes
  print(dif)
  
  if(dif > 0):
    for i in range(dif):
      add_stop_word(unique_words[i])
  
  df_new = []
  for i in range(len(df)):
    X, Y = df[i]
    X = ' '.join(x for x in X.split(' ') if nlp.vocab[x].is_stop == False)
    X = re.sub(' +', ' ', X)
    
    if(len(X) != 0):
      df_new.append([X, Y])

  return n_i, df_new

In [ ]:
# n_i, df_names = get_filtered_df(df_names)

In [ ]:
def bag_of_words(data):
  bag_of_words = []
  for text, spam in data:
    words = text.split(" ")
    new_text = ""
    for word in words:
      if nlp.vocab[word].is_stop:
        pass
      else:
        new_text += word + " "
    bag_of_words.append([new_text, spam])
  
  bag_of_words_tokens = []
  for text, spam in data:
    tokens = word_tokenize(text, 'spanish')
    tokens = [w.lower() for w in tokens if w.isalpha() and nlp.vocab[w].is_stop == False]
    bag_of_words_tokens.append([tokens, spam])

  tokens = {}
  for text, spam in data:
    words = word_tokenize(text, 'spanish')
    words = [w.lower() for w in words if w.isalpha() and nlp.vocab[w].is_stop == False]
    for w in words:
      if w.isalpha() and nlp.vocab[w].is_stop == False:
        if w not in tokens:
          tokens[w] = 0
        if w in tokens:
          tokens[w] += 1
  
  keys = list(tokens.keys())

  bag_of_words = []
  quantity = []
  for text, spam in data:
    words = word_tokenize(text, 'spanish')
    words = [w.lower() for w in words if w.isalpha() and nlp.vocab[w].is_stop == False]
    bag = [0] * len(tokens)
    q = [0] * len(tokens)
    for i in range(len(words)):
      word = words[i]
      if word in keys:
        index = keys.index(word)
        bag[index] = 1
        q[index] += 1
    bag_of_words.append(bag)
    quantity.append(q)

  header_generated = []
  for key in keys:
    header_generated.append(key)
  header_generated.append('Etiquetado')
  data_generated = []
  for i in range(len(bag_of_words)):
    e = []
    for b in bag_of_words[i]:
      e.append(b == 1)
    e.append(bag_of_words_tokens[i][1])
    data_generated.append(e)
  
  n_i = len(data_generated)
  names = header_generated
  df_names = data_generated

  db = dataF(n_i, names, df_names)

  return db

In [ ]:
def get_transponse(edges):
  n = len(edges)
  aux = [[] for _ in range(n)]
  for i in range(n):
    for x in edges[i]:
      aux[x].append(i)
  
  return aux

def get_topo_sort(edges):
  
  trans = get_transponse(edges)

  def dfs(x, visited, ts):
    if(visited[x]):
      return
    
    visited[x] = True
    for y in trans[x]:
      dfs(y, visited, ts)
    
    ts.append(x)

  n = len(trans)
  visited = [False for i in range(n)]
  ts = []

  for i in range(n):
    if(not visited[i]):
      dfs(i, visited, ts)
  
  return ts[::-1]

def get_matrix_factor(cols, df):
  uniqs = []
  cnt = []
  ids = []
  vuelta = []
  total = 1

  for x in cols:

      uniqs.append([i for i in range(df.card[x])])
      ids.append(total)
          
      total*= df.card[x]
      
      cnt.append(0)
      vuelta.append(0)
      
  M = []

  for u in range(total):
      vals = []
      i = 0
      for x in cols:
          vals.append([x, uniqs[i][cnt[i]]])
          i+= 1
      
      M.append( [e[1] for e in vals] + [1])

      for j in range(len(cols)):
          vuelta[j]+= 1
          if(vuelta[j] == ids[j]):
              cnt[j] = (cnt[j] + 1)%len(uniqs[j])
              vuelta[j] = 0
  
  return M

def columns_order(c1, c2):
  c3 = []
  for i in range(1, len(c1)):
    c3.append((c1[i], c2[i]))

  c3.sort(key = lambda x: x[0])
  
  for i in range(1, len(c1)):
    c1[i], c2[i] = c3[i-1]

def get_sum_matrix(map_cols,mult_fact,row,new_cols,k):
  sum = 0
  for ROW in mult_fact:
    NR = ROW[:-1]
    NR.pop(map_cols[k])
    if(NR == row):
      sum+= ROW[-1]

  return sum

def hash_list_2(c1, c2):
  return "cols: " + str(c1) + " | vals: " + str(c2) 

def get_val_fact_2(map, cols, vals):
  return map[hash_list_2(cols, vals)] 

def query(map, df, ev, ev_val, q, orden=[]):

  def get_unique_cols(mx):
    st = set()
    for x in mx:
      for elem in x:
        st.add(elem)

    st = list(st)
    st.sort()

    return st
  
  if(len(orden) == 0):
    orden = get_topo_sort(df.lin)
  
  n = len(orden)
  orden = [x for x in orden if x not in ev and x not in q]

  factor_names = []
  added_factor = []

  map_factores = dict()
  for i in range(n):
    factor_names.append([i] + df.lin[i])
  for k in orden: #topo sort con ocultas
    
    rem = [x for x in factor_names if k in x]
    arem = [x for x in added_factor if k in x]

    cols = get_unique_cols(rem + arem)
    map_cols = dict()

    for i in range(len(cols)):
      map_cols[cols[i]] = i


    mult_fact = get_matrix_factor(cols, df)
 
    # multiplicacion
    for row in mult_fact:
      for fact in rem:
        vals_ = []
        for f in fact:
          vals_.append(row[map_cols[f]])
    
        row[-1]*= get_val_fact(map,fact,vals_)
      
      for fact in arem:
        vals_ = []
        for f in fact:
          vals_.append(row[map_cols[f]])
    
        row[-1]*= get_val_fact_2(map_factores,fact,vals_)
    
    
    #suma
    new_cols = cols.copy()
    new_cols.remove(k)
    new_mult_fact = get_matrix_factor(new_cols, df)

    for row in new_mult_fact:
      row[-1] = get_sum_matrix(map_cols,mult_fact,row[:-1],new_cols,k)


    #agregar nuevo factor
    if(new_cols):
      added_factor.append(new_cols)
      for x in new_mult_fact:
        aux = hash_list_2(new_cols, x[:-1])
        if(aux in map_factores):
          map_factores[aux]*= x[-1]
        else:
          map_factores[aux] = x[-1]

    for x in rem:
      factor_names.remove(x)
    
    for x in arem:
      added_factor.remove(x)



  # multiplicaion final y argmax para la query
  cols = get_unique_cols(factor_names + added_factor)
  map_cols = dict()

  for i in range(len(cols)):
    map_cols[cols[i]] = i

  k = q[0]
  id_col = map_cols[k]
  card = df.card[k]

  FF = get_matrix_factor(cols, df)

  for row in FF:
    for fact in factor_names:
      vals_ = []
      for f in fact:
        vals_.append(row[map_cols[f]])
      
      row[-1]*= get_val_fact(map,fact,vals_)
    
    for fact in added_factor:
      vals_ = []
      for f in fact:
        vals_.append(row[map_cols[f]])
    
      row[-1]*= get_val_fact_2(map_factores,fact,vals_)

  # argmax


  arg = dict()

  for i in range(card):
    arg[i] = 0

  # print(cols)

  for row in FF:
    ok = 1
    for i in range(len(ev)):
      c_ = map_cols[ev[i]]
      if(row[c_] != ev_val[i]):
        ok = 0
    
    if(ok):
      # print(row)
      arg[row[id_col]]+= row[-1]
  
  print("-----------------")

  maxargs = [None, 0]
  for x in arg:
    if(arg[x] > maxargs[1]):
      maxargs = [x,arg[x]]

  print("maxargs: ", maxargs)

  return arg, maxargs

In [ ]:
def predict_query(d, test):
  print(test)

  test = process_text(test)

  ev = []
  ev_val = []
  
  test = test.split(' ') 

  print(test)

  for i in range(len(test)):
    if(test[i] in d.names):
      ev.append(d.mnames[test[i]])
      ev_val.append(1)
  
  if(len(ev) == 0):
    return {0 : 0.3, 1 : 0.3, 2 : 0.3} , [d.mcvals[len(d.names) - 1]['Neutro'], 0.3] 

  return query(d.mapFact, d, ev, ev_val, [ d.mnames["Etiquetado"] ])  
  

In [ ]:
# df_names_len = len(df_names)
# cut = int(df_names_len * 0.7)
# train = df_names[0:cut]
# test = df_names[cut:df_names_len]
# db = bag_of_words(train)

Aprendizaje Supervisado y PLN:] se debe entrenar redes bayesianas generadas usando (1) hill-climbing/greedy, (2) b´usqueda K2 con la m´etrica entrop´ıa y m´etrica
K2 con vectorizaci´on bag-of-words. Aplicar las m´etricas de desempe˜no vistas en clase:
F1, precisi´on, recall, accuracy, gr´aficos ROC y plasmar conclusiones al respecto.

In [ ]:
# db_k2 = K2(db, 4, 5, 'K2')

In [ ]:
# predict_query(db_k2, 'fiore')

In [ ]:
# x = predict_query(db_k2, 'gracias gustar')

In [ ]:
def getPredicts(db, test):
  data_test_Y = []
  data_real = []

  for elem in test:
    x = predict_query(db, elem[0])
    data_test_Y.append(x[1])
    data_real.append(elem[1])

  return data_real, data_test_Y

In [ ]:
def divide(n, d):
    return n / d if d else 0

In [ ]:
class metrics:
  def __init__(self, data_test,data_pred,name="auc.png"):

    for i in range(len(data_test)):
      if data_test[i] == 'Negativo':
        data_test[i] = 2
      elif data_test[i] == 'Positivo':
        data_test[i] = 0
      elif data_test[i] == 'Neutral':
        data_test[i] = 1

    self.scores = data_pred
    self.real = data_test

    self.N = data_test.count(2)
    self.Ntl = data_test.count(1)
    self.P = data_test.count(0)

    self.TP = 0
    self.FP = 0

    self.TN = 0
    self.FN = 0

    self.TNtl = 0
    self.FNtl = 0

    self.name = name

    for i in range(len(data_test)):
      if data_test[i] == 0 and data_pred[i][0] == 0:
        self.TP += 1
      elif data_test[i] != 0 and data_pred[i][0] == 0:
        self.FP += 1
      elif data_test[i] != 2 and data_pred[i][0] == 2:
        self.FN += 1
      elif data_test[i] == 2 and data_pred[i][0] == 2:
        self.TN += 1
      elif data_test[i] != 1 and data_pred[i][0] == 1:
        self.FNtl += 1
      elif data_test[i] == 1 and data_pred[i][0] == 1:
        self.TNtl += 1

  def __str__(self):
    string = """
    P: {}
    N: {}
    TP: {}
    FP: {}
    TN: {}
    FN: {}
    """.format(self.P,self.N,self.TP,self.FP,self.TN,self.TN)
    return string

  def precision_score(self,clase):
    if clase == 0:
      return divide(self.TP,(self.FP+self.TP))
    elif clase == 2:
      return divide(self.TN,(self.FN+self.TN))
    elif clase == 1:
      return divide(self.TNtl,(self.FNtl+self.TNtl))
  
  def recall_score(self,clase):
    if clase == 0:
      return divide(self.TP,(self.FN+self.FNtl+self.TP))
    elif clase == 2:
      return divide(self.TN,(self.FP+self.FNtl+self.TN))
    elif clase == 1:
      return divide(self.TNtl,(self.FN+self.FP+self.TNtl))

  def accuracy_score(self):
    return divide((self.TP+self.TN+self.TNtl),(self.TP+self.TN+self.FN+self.FP+self.FNtl+self.TNtl))

  def f1_score(self,clase):
    return divide(2*self.precision_score(clase)*self.recall_score(clase),(self.precision_score(clase)+self.recall_score(clase)))
    
  def roc_curve(self):
    import numpy as np
    import matplotlib.pyplot as plt
    plt.style.use('ggplot')
    plt.rcParams["figure.figsize"] = (16,9)
    %matplotlib inline

    FPR = []
    TPR = []

    thresholds = np.arange(0.0, 1.01, 0.2)
    for t in thresholds:
      FP_ = 0
      TP_ = 0
      t = round(t,2)
      for i in range(len(self.scores)):
        if(self.scores[i][1] >= t):
          if self.real[i] == 0:
            TP_ += 1
          if self.real[i] != 0:
            FP_ += 1
      FPR.append(divide(FP_,self.N))
      TPR.append(divide(TP_,self.P))
    
    auc = -1 * np.trapz(TPR, FPR)

    plt.plot(FPR, TPR, linestyle='--', marker='o', color='darkorange', lw = 2, label='ROC curve', clip_on=False)
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve, AUC = %.2f'%auc)
    plt.legend(loc="lower right")
    plt.savefig(self.name)

In [ ]:
from collections import Counter

def ensemble(redes, instancia, tipo):
  moda = []
  media = []

  for red in redes:
    args, mx = predict_query(red, instancia)
    moda.append(mx[0])
    media.append(args)
  
  c = Counter(moda)
  
  args_med = {}
  for key in media[0].keys():
    args_med[key] = 0

  for elem in media:
    for arg in elem:
      args_med[arg]+= elem[arg]


  for elem in args_med:
    args_med[elem]/= len(redes)
  
  maxargs = [None, 0]
  for x in args_med:
    if(args_med[x] > maxargs[1]):
      maxargs = [x,args_med[x]]
  
  
  if(tipo == 'Moda'):
    return [ c.most_common(1)[0][0] , args_med[c.most_common(1)[0][0]]] 
            
  return maxargs

In [ ]:
def predictEnsemble(redes, tipo, test):
  data_test_Y = []
  data_pred = []

  for elem in test:
    x = ensemble(redes, test, tipo)
    data_test_Y.append(x)
  
  return data_pred, data_test_Y

In [ ]:
!pip install graphviz
from graphviz import Digraph
import ipywidgets as widgets
from IPython.display import display
import graphviz

In [ ]:
tab_nest = widgets.Tab()

loaded = False
uploaded = False

upl_data = widgets.FileUpload(accept='.xlsx',multiple=False,description="Select data.xlsx")

btn_upl_data = widgets.Button(
    description='Upload DATASET',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
    )

input_limit = widgets.Text(
    value='50',
    placeholder='Cantidad a leer',
    description='Cantidad a leer:',
    disabled=False
)

btn_create = widgets.Button(
    description='CREATE COMPONENTS',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check',
    layout=widgets.Layout(width="17%")
    )

def upl_data_file(a):
  global uploaded
  global upl_data
  if upl_data.value == {}:
    print("No hay ningun archivo subido.")
  else:
    uploaded_filename = next(iter(upl_data.value))
    content = upl_data.value[uploaded_filename]['content']
    with open('data.xlsx', 'wb') as f: f.write(content)
    uploaded = True
    print("Archivo subido")
    upl_data.value.clear()
    upl_data._counter = 0

def load_components(a):
    global uploaded
    global db
    global tab_nest
    global input_limit

    if uploaded == False:
        print("Suba un archivo para crear los componentes")
        return

    limit = int(input_limit.value)
    
    # SHOW DATAFRAME
    n_i, names, df_names = readFile('data.xlsx', limit)
    n_i, df_names = get_filtered_df(df_names, n_i)
    df_names_len = len(df_names)
    cut = int(df_names_len * 0.7)
    train = df_names[0:cut]
    test = df_names[cut:df_names_len]
    print("Creando")
    db = bag_of_words(train)
    print("Creado")

    rows_boxes = list()
    labels=list()
    for head in db.names:
      label = widgets.Button(description = head, disabled = True, button_style='info')
      labels.append(label)
    headers_box = widgets.HBox(labels)

    for row in db.df_names:
      values=list()
      for val in row:
        value = widgets.Button(description = str(val), disabled = True)
        values.append(value)
      rows_boxes.append(widgets.HBox(values))
    dataset_box = widgets.VBox([headers_box,*rows_boxes])

    #GREEDY ENTROPIA
    db_ge = greedyLS(db, 'Entropy')
    metrics_ge = list()
    graph_ge = list()

    print("Predictions GE")
    ge_test_y, ge_data_pred = getPredicts(db_ge, test)
    print("END GE")
    history_ge = metrics(ge_test_y,ge_data_pred,name="ge_roc.png")
    precision_ge = str(history_ge.precision_score(0))
    recall_ge = str(history_ge.recall_score(0))
    accuracy_ge = str(history_ge.accuracy_score())
    f1_ge = str(history_ge.f1_score(0))
    history_ge.roc_curve()

    precison_ge_widget = widgets.Text(
      value=precision_ge,
      description='Precision:',
      disabled=True
    )

    recall_ge_widget = widgets.Text(
      value=recall_ge,
      description='Recall:',
      disabled=True
    )
        
    accuracy_ge_widget = widgets.Text(
      value=accuracy_ge,
      description='Accuracy:',
      disabled=True
    )

    f1_ge_widget = widgets.Text(
      value=f1_ge,
      description='Score:',
      disabled=True
    )

    file_image_ge_roc = open("ge_roc.png", "rb")
    image_ge_roc = file_image_ge_roc.read()
    image_widget_ge_roc = widgets.Image(value=image_ge_roc, format='png', layout=widgets.Layout(width='600px'))

    metrics_ge.append(precison_ge_widget)
    metrics_ge.append(recall_ge_widget)
    metrics_ge.append(accuracy_ge_widget)
    metrics_ge.append(f1_ge_widget)
    metrics_ge.append(image_widget_ge_roc)
    left_ge_vbox = widgets.VBox(metrics_ge)

    dot_ge = Digraph(format='png')
    for head in db_ge.names:
      dot_ge.node(head)
    for edge in db_ge.edges:
      parent = edge[0] 
      child = edge[1]
      dot_ge.edge(parent, child)
    dot_ge.render("ge")
    file_image_ge = open("ge.png", "rb")
    image_ge = file_image_ge.read()
    image_widget_ge = widgets.Image(value=image_ge, format='png', layout=widgets.Layout(width='1200px'))
    graph_ge.append(image_widget_ge)
    right_ge_vbox = widgets.VBox(graph_ge)
    tab_ge = widgets.HBox([left_ge_vbox, right_ge_vbox])

    #K2 ENTROPIA
    db_ke = K2(db, 2, 5, 'Entropy')
    metrics_ke = list()
    graph_ke = list()

    print("Predictions KE")
    ke_test_y, ke_data_pred = getPredicts(db_ke, test)
    print("KE")

    history_ke = metrics(ke_test_y,ke_data_pred,name="ke_roc.png")
    precision_ke = str(history_ke.precision_score(0))
    recall_ke = str(history_ke.recall_score(0))
    accuracy_ke = str(history_ke.accuracy_score())
    f1_ke = str(history_ke.f1_score(0))
    history_ke.roc_curve()

    precison_ke_widget = widgets.Text(
      value=precision_ke,
      description='Precision:',
      disabled=True
    )

    recall_ke_widget = widgets.Text(
      value=recall_ke,
      description='Recall:',
      disabled=True
    )
        
    accuracy_ke_widget = widgets.Text(
      value=accuracy_ke,
      description='Accuracy:',
      disabled=True
    )

    f1_ke_widget = widgets.Text(
      value=f1_ke,
      description='Score:',
      disabled=True
    )

    file_image_ke_roc = open("ke_roc.png", "rb")
    image_ke_roc = file_image_ke_roc.read()
    image_widget_ke_roc = widgets.Image(value=image_ke_roc, format='png', layout=widgets.Layout(width='600px'))

    metrics_ke.append(precison_ke_widget)
    metrics_ke.append(recall_ke_widget)
    metrics_ke.append(accuracy_ke_widget)
    metrics_ke.append(f1_ke_widget)
    metrics_ke.append(image_widget_ke_roc)
    left_ke_vbox = widgets.VBox(metrics_ke)

    dot_ke = Digraph(format='png')
    for head in db_ke.names:
      dot_ke.node(head)
    for edge in db_ke.edges:
      parent = edge[0] 
      child = edge[1]
      dot_ke.edge(parent, child)
    dot_ke.render("ke")
    file_image_ke = open("ke.png", "rb")
    image_ke = file_image_ke.read()
    image_widget_ke = widgets.Image(value=image_ke, format='png', layout=widgets.Layout(width='1200px'))
    graph_ke.append(image_widget_ke)
    right_ke_vbox = widgets.VBox(graph_ke)
    tab_ke = widgets.HBox([left_ge_vbox, right_ke_vbox])

    #GREEDY K2
    db_gk = greedyLS(db, 'K2')
    metrics_gk = list()
    graph_gk = list()

    print("Predictions GK")
    gk_test_y, gk_data_pred = getPredicts(db_gk, test)
    print("END GK")

    history_gk = metrics(gk_test_y,gk_data_pred,name="gk_roc.png")
    precision_gk = str(history_gk.precision_score(0))
    recall_gk = str(history_gk.recall_score(0))
    accuracy_gk = str(history_gk.accuracy_score())
    f1_gk = str(history_gk.f1_score(0))
    history_gk.roc_curve()

    precison_gk_widget = widgets.Text(
      value=precision_gk,
      description='Precision:',
      disabled=True
    )

    recall_gk_widget = widgets.Text(
      value=recall_gk,
      description='Recall:',
      disabled=True
    )
        
    accuracy_gk_widget = widgets.Text(
      value=accuracy_gk,
      description='Accuracy:',
      disabled=True
    )

    f1_gk_widget = widgets.Text(
      value=f1_gk,
      description='Score:',
      disabled=True
    )

    file_image_gk_roc = open("gk_roc.png", "rb")
    image_gk_roc = file_image_gk_roc.read()
    image_widget_gk_roc = widgets.Image(value=image_gk_roc, format='png', layout=widgets.Layout(width='600px'))

    metrics_gk.append(precison_gk_widget)
    metrics_gk.append(recall_gk_widget)
    metrics_gk.append(accuracy_gk_widget)
    metrics_gk.append(f1_gk_widget)
    metrics_gk.append(image_widget_gk_roc)
    left_gk_vbox = widgets.VBox(metrics_gk)

    dot_gk = Digraph(format='png')
    for head in db_gk.names:
      dot_gk.node(head)
    for edge in db_gk.edges:
      parent = edge[0] 
      child = edge[1]
      dot_gk.edge(parent, child)
    dot_gk.render("gk")
    file_image_gk = open("gk.png", "rb")
    image_gk = file_image_gk.read()
    image_widget_gk = widgets.Image(value=image_gk, format='png', layout=widgets.Layout(width='1200px'))
    graph_gk.append(image_widget_gk)
    right_gk_vbox = widgets.VBox(graph_gk)
    tab_gk = widgets.HBox([left_ge_vbox, right_gk_vbox])

    #K2 K2
    db_kk = K2(db, 2, 5, 'K2')
    metrics_kk = list()
    graph_kk = list()

    print("Predictions KK")
    kk_test_y, kk_data_pred = getPredicts(db_kk, test)
    print("END KK")

    history_kk = metrics(kk_test_y,kk_data_pred,name="kk_roc.png")
    precision_kk = str(history_kk.precision_score(0))
    recall_kk = str(history_kk.recall_score(0))
    accuracy_kk = str(history_kk.accuracy_score())
    f1_kk = str(history_kk.f1_score(0))
    history_kk.roc_curve()

    precison_kk_widget = widgets.Text(
      value=precision_kk,
      description='Precision:',
      disabled=True
    )

    recall_kk_widget = widgets.Text(
      value=recall_kk,
      description='Recall:',
      disabled=True
    )
        
    accuracy_kk_widget = widgets.Text(
      value=accuracy_kk,
      description='Accuracy:',
      disabled=True
    )

    f1_kk_widget = widgets.Text(
      value=f1_kk,
      description='Score:',
      disabled=True
    )

    file_image_kk_roc = open("kk_roc.png", "rb")
    image_kk_roc = file_image_kk_roc.read()
    image_widget_kk_roc = widgets.Image(value=image_kk_roc, format='png', layout=widgets.Layout(width='600px'))

    metrics_kk.append(precison_kk_widget)
    metrics_kk.append(recall_kk_widget)
    metrics_kk.append(accuracy_kk_widget)
    metrics_kk.append(f1_kk_widget)
    metrics_kk.append(image_widget_kk_roc)
    left_kk_vbox = widgets.VBox(metrics_kk)

    dot_kk = Digraph(format='png')
    for head in db_kk.names:
      dot_kk.node(head)
    for edge in db_kk.edges:
      parent = edge[0] 
      child = edge[1]
      dot_kk.edge(parent, child)
    dot_kk.render("kk")
    file_image_kk = open("kk.png", "rb")
    image_kk = file_image_kk.read()
    image_widget_kk = widgets.Image(value=image_kk, format='png', layout=widgets.Layout(width='1200px'))
    graph_kk.append(image_widget_kk)
    right_kk_vbox = widgets.VBox(graph_kk)
    tab_kk = widgets.HBox([left_ge_vbox, right_kk_vbox])

    redes = [db_ge, db_ke, db_gk, db_kk]

    #E M
    metrics_em = list()

    print("Predictions EM")
    em_test_y, em_data_pred = predictEnsemble(redes,'Moda',test)
    print("END EM")

    history_em = metrics(em_test_y,em_data_pred,name="em_roc.png")
    precision_em = str(history_em.precision_score(0))
    recall_em = str(history_em.recall_score(0))
    accuracy_em = str(history_em.accuracy_score())
    f1_em = str(history_em.f1_score(0))

    precison_em_widget = widgets.Text(
      value=precision_em,
      description='Precision:',
      disabled=True
    )

    recall_em_widget = widgets.Text(
      value=recall_em,
      description='Recall:',
      disabled=True
    )
        
    accuracy_em_widget = widgets.Text(
      value=accuracy_em,
      description='Accuracy:',
      disabled=True
    )

    f1_em_widget = widgets.Text(
      value=f1_em,
      description='Score:',
      disabled=True
    )

    metrics_em.append(precison_em_widget)
    metrics_em.append(recall_em_widget)
    metrics_em.append(accuracy_em_widget)
    metrics_em.append(f1_em_widget)
    left_em_vbox = widgets.VBox(metrics_em)

    right_em_vbox = widgets.VBox()
    tab_em = widgets.HBox([left_ge_vbox, right_em_vbox])   

        #E M
    metrics_em = list()

    print("Predictions EM")
    em_test_y, em_data_pred = predictEnsemble(redes,'Media',test)
    print("END EM")

    history_em = metrics(em_test_y,em_data_pred,name="em_roc.png")
    precision_em = str(history_em.precision_score(0))
    recall_em = str(history_em.recall_score(0))
    accuracy_em = str(history_em.accuracy_score())
    f1_em = str(history_em.f1_score(0))

    precison_em_widget = widgets.Text(
      value=precision_em,
      description='Precision:',
      disabled=True
    )

    recall_em_widget = widgets.Text(
      value=recall_em,
      description='Recall:',
      disabled=True
    )
        
    accuracy_em_widget = widgets.Text(
      value=accuracy_em,
      description='Accuracy:',
      disabled=True
    )

    f1_em_widget = widgets.Text(
      value=f1_em,
      description='Score:',
      disabled=True
    )

    metrics_em.append(precison_em_widget)
    metrics_em.append(recall_em_widget)
    metrics_em.append(accuracy_em_widget)
    metrics_em.append(f1_em_widget)
    left_em_vbox = widgets.VBox(metrics_em)

    right_em_vbox = widgets.VBox()
    tab_ex = widgets.HBox([left_ge_vbox, right_em_vbox])   

    tab_nest.children = [*tab_nest.children,dataset_box,tab_ge,tab_ke,tab_gk,tab_kk,tab_em,tab_ex]

btn_upl_data.on_click(upl_data_file)
btn_create.on_click(load_components)

upl_data_box = widgets.HBox([upl_data,btn_upl_data, input_limit])
upl_box = widgets.VBox([upl_data_box,btn_create])

tab_nest.children = [upl_box]
tab_nest.set_title(0, 'UPLOAD')
tab_nest.set_title(1, 'DATASET')
tab_nest.set_title(2, 'GREEDY/ENTROPIA')
tab_nest.set_title(3, 'K2/ENTROPIA')
tab_nest.set_title(4, 'GREEDY/K2')
tab_nest.set_title(5, 'K2/K2')
tab_nest.set_title(6, 'MODA/ENSAMBLE')
tab_nest.set_title(6, 'MAX/ENSAMBLE')
tab_nest

Archivo subido
['administracion', 'hermano', 'nosotras', 'tambien', 'robaron', 'deberian', 'opcion', 'sus', 'eliminen', 'injusto', 'ciclo', 'wapaax', 'siento', 'estamos', 'robotico', 'verlas', 'trabajo', 'gordita', 'como', 'ps', 'esperamos', 'seria', 'pre', 'fechas', 'capo', 'sera', 'muchas', 'tarde', 'colaboradores', 'pagina', 'excelente', 'soy', 'mundo', 'v', 'esta', 'solo', 'caso', 'todas', 'puedes', 'salgo', 'sentimos', 'y', 'invento', 'wow', 'espectacular', 'pasando', 'acercarte', 'visto', 'tu', '', 'son', 'pero', 'ustedes', 'este', 'tendran', 'acreditar', 'tremendo', 'publiquen', 'eres', 'al', 'orgulloupn', 'danza', 'tener', 'salieron', 'mal', 'profesionales', 'siga', 'el', 'gran', 'manita', 'seguro', 'upc', 'alexis', 'ahora', 'elenco', 'etiquetenlas', 'gracias', 'carreras', 'cuando', 'sabes', 'inscripcion', 'pruebas', 'mas', 'universidad', 'si', 'vvv', 'genial', 'me', 'upn', 'ti', 'burlados', 'por', 'pago', 'que', 'la', 'fecha', 'virtuales', 'sean', 'campus', 'conscientes', 'eow